In [ ]:
#This is a Naive Bayes Model and K-NEarest Neighbor
#It can only be run on binary data
#However, it is set up to run on unsorted binary data

In [6]:
import numpy as np
import matplotlib
%matplotlib inline
from matplotlib import pyplot as plt
import pandas as pd
import random as random
from csv import reader
from math import sqrt
from math import exp
from math import pi
import itertools
print("Done")

Done


In [7]:
#Pull in the dataset
#Change file location, as needed
#Also for the file you are pulling in
#train = pd.read_csv("C:/Users/Veteran/Documents/Luke's School/Terms/Fall 2019/CS 541/Homework 3/orig_train.txt", delimiter = "\s+", header = None)
#test = pd.read_csv("C:/Users/Veteran/Documents/Luke's School/Terms/Fall 2019/CS 541/Homework 3/orig_test.txt", delimiter = "\s+", header = None)
#train = pd.read_csv("C:/Users/Veteran/Documents/Luke's School/Terms/Fall 2019/CS 541/Homework 3/itg_train.txt", delimiter = "\s+", header = None)
#test = pd.read_csv("C:/Users/Veteran/Documents/Luke's School/Terms/Fall 2019/CS 541/Homework 3/itg_test.txt", delimiter = "\s+", header = None)
train = pd.read_csv("C:/Users/Veteran/Documents/Luke's School/Terms/Fall 2019/CS 541/Homework 3/resplit_train.txt", delimiter = "\s+", header = None)
test = pd.read_csv("C:/Users/Veteran/Documents/Luke's School/Terms/Fall 2019/CS 541/Homework 3/resplit_test.txt", delimiter = "\s+", header = None)

In [8]:
#These will apply to both sets of values
train = np.asarray(train).astype(int)
test = np.asarray(test).astype(int)
ncol_train = len(train[0])
ncol_test = len(test[0])
nrows_train = len(train)
nrows_test = len(test)
abnormal = np.zeros((1,ncol_train))
normal = np.zeros((1,ncol_train))
print(ncol_train)

23


In [9]:
#These are necessary functions
#They can be used for both Naive Bayes
def average(data):
    mu = sum(data)/float(len(data))
    return mu

In [10]:
#First we need to point out the column of classification
#Again AI is axis = 0 and ML is axis = -1

#Artificial Intelligence Homework
#Get the minimum and maximum values of the classes
maxi = train.max(axis = 0)[0]
mini = train.min(axis = 0)[0]

In [11]:
for j in range(mini, maxi):
    for i in range(nrows_train):
        if train[i,0] == mini:
            stack = train[i,]
            abnormal = np.vstack([abnormal, stack])
        if train[i,0] == maxi:
            stack1 = train[i,]
            normal = np.vstack([normal,stack1])
abnormal = np.delete(abnormal, 0, axis = 0)
normal = np.delete(normal, 0, axis = 0)

In [12]:
abnormal_noclass = np.delete(abnormal, 0, axis = 1)
normal_noclass = np.delete(normal, 0, axis = 1)

In [13]:
#Remove first value, since that is the classification
mean_abnormal = average(abnormal_noclass)
std_abnormal = np.std(abnormal_noclass, axis = 0)
mean_normal = average(normal_noclass)
std_normal = np.std(normal_noclass, axis = 0)

In [14]:
#We need to calculate the probability that a column is zero given abnormal results
#And one given abnormal results
#Then we have to do the same for normal results
count0, count1 = [],[]
for k in range(mini, maxi):
    
    for i in range(len(abnormal_noclass[0])):
        count00,count11 = 0,0
        for j in range(len(abnormal_noclass)):
            if abnormal_noclass[j,i] == mini:
                count00 +=1
            else:
                count11 += 1
        count0.append(count00)
        count1.append(count11)

In [15]:
#We need to calculate the probability that a column is zero given abnormal results
#And one given abnormal results
#Then we have to do the same for normal results
count0n, count1n = [],[]
for k in range(mini, maxi):
    
    for i in range(len(normal_noclass[0])):
        count00n,count11n = 0,0
        for j in range(len(normal_noclass)):
            if normal_noclass[j,i] == mini:
                count00n +=1
            else:
                count11n += 1
        count0n.append(count00n)
        count1n.append(count11n)

In [16]:
#Redefine this variable
ab_giv_1 = np.asarray(count1)
ab_giv_0 = np.asarray(count0)
norm_giv_1 = np.asarray(count1n)
norm_giv_0 = np.asarray(count0n)
print(ab_giv_1)
print(ab_giv_0)
print(norm_giv_1)
print(norm_giv_0)

[10  4  8  6  8  2  5  7  7  7  3  6  6  6  2  3  0  0  5  5  5  6]
[26 32 28 30 28 34 31 29 29 29 33 30 30 30 34 33 36 36 31 31 31 30]
[67 38 60 45 61 34 40 69 48 60 39 46 81 45 26 50 23 24 39 51 56 64]
[ 74 103  81  96  80 107 101  72  93  81 102  95  60  96 115  91 118 117
 102  90  85  77]


In [17]:
#Since we have already separated each class
#We already have the given values for each column
#Therefore, we need to divide by the total number of values
#In the training data, we have 40 abnormal and 40 normal instances
#Due to this, we need to divide each count by 40.
#However, we don't know that it will always be that way, so instead
#We will divide by the length of abnormal and normal
#Since that can change in the future, we will use the row length
#Of the normal and abnormal arrays
nrows_norm = len(normal)
nrows_ab = len(abnormal)
prob_ab_giv_0 = ab_giv_0/nrows_ab#probability column is 0 given abnormal
prob_ab_giv_1 = ab_giv_1/nrows_ab#probability column is 1 given abnormal
prob_norm_giv_0 = norm_giv_0/nrows_norm#probability column is 0 given normal
prob_norm_giv_1 = norm_giv_1/nrows_norm#probability column is 1 given normal

In [18]:
prob_abnormal = nrows_ab/nrows_train#probability of getting abnormal results
prob_normal = nrows_norm/nrows_train#probabiliy of getting normal results

In [19]:
#Now that we have gotten all the probabilities that we need
#We can define the naive bayes function and run it through the test set. 
#This model is not trained in the usual fashion
#So, once we have the necessary probabilities
#We really just need to calculate the probability 
#Of each column on the test set.
nrows_test = len(test)
#We know the number of columns in the training has to be the same as those 
#In the test set, so we don't need to calculate the number of columns
test_class = test[:,0]#Set up the classifications
test = np.delete(test, 0, axis = 1)#Remove the classification column

In [20]:
pred = np.array([])
for i in range(nrows_test):
    ab_prob_vector = np.array([])
    norm_prob_vector = np.array([])
    for j in range(len(test[0])):
        if test[i,j] == mini:
            ab_prob_vector = np.append(prob_ab_giv_0[j] ,ab_prob_vector)
            norm_prob_vector = np.append(prob_norm_giv_0[j], norm_prob_vector)
            print(prob_ab_giv_0.shape)
        else:
            ab_prob_vector = np.append(prob_ab_giv_1[j], ab_prob_vector)
            norm_prob_vector = np.append(prob_norm_giv_1[j], norm_prob_vector)
    ab_pred = np.prod(ab_prob_vector)*prob_abnormal
    norm_pred = np.prod(norm_prob_vector)*prob_normal
    if ab_pred == norm_pred:
        pred = np.append(0, pred)
        #pred = np.append(random.randint(0, 1), pred)
    if ab_pred > norm_pred:
        pred = np.append(0, pred)
    if ab_pred < norm_pred:
        pred = np.append(1, pred)

(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,)
(22,

In [154]:
acc = 0
pos = 0
neg = 0
num_neg = 0
num_pos = 0
for i in range(len(pred)):
    if test_class[i] == 0:
        num_pos += 1
    if test_class[i] == 1:
        num_neg += 1
    if pred[i] == test_class[i]:
        acc += 1
    if pred[i] == 0 and test_class[i] == 0:
        pos += 1
    if pred[i] == 1 and test_class[i] == 1:
        neg += 1

In [155]:
data = "orig"
data1 = "itg"
data2 = "resplit"
accuracy = acc/len(test)
t_pos = pos/num_pos
t_neg = neg/num_neg
print("For Naive Bayes:")
print(data2, acc,"/",len(test), "(",accuracy,")", pos,"/",num_pos, "(",t_pos,")", neg,"/",num_neg, "(",t_neg,")")

For Naive Bayes:
resplit 48 / 90 ( 0.5333333333333333 ) 1 / 19 ( 0.05263157894736842 ) 47 / 71 ( 0.6619718309859155 )


In [156]:
#This is a second model used to classify Heart Anomalies
#This one will be a k-nearest neighbor
#We will see the classification accuracy 
#Of this model compared to the naive bayes

In [157]:
#To determine distances we need to use
#Euclidean Distance
def euc_dist(row1, row2):
    dist = sum(np.subtract(row1,row2)**2)
    dist = sqrt(dist)
    return dist 

In [158]:
knn_class = np.array([])
for i in range(nrows_test):
    ab_dist = np.array([])
    norm_dist = np.array([])
    for j in range(len(abnormal)):
        for k in range(len(normal)):
            new_ab_dist = euc_dist(test[i], abnormal_noclass[j])
            ab_dist = np.append(new_ab_dist, ab_dist)
            new_norm_dist = euc_dist(test[i], normal_noclass[k])
            norm_dist = np.append(new_norm_dist, norm_dist)
    ab_min = np.amin(ab_dist)
    norm_min = np.amin(norm_dist) 
    if ab_min == norm_min:
        knn_class = np.append(0, knn_class)
        #knn_class = np.append(random.randint(0,1), knn_class)
    if ab_min > norm_min:
        knn_class = np.append(0, knn_class)
    if ab_min < norm_min:
        knn_class = np.append(1, knn_class)
print(knn_class)
print(knn_class.shape)
print(test_class)
print(test_class.shape)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(90,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
(90,)


In [159]:
knn_acc = 0
knn_pos = 0
knn_neg = 0
knn_num_neg = 0
knn_num_pos = 0
for l in range(len(knn_class)):
    if test_class[l] == 0:
        knn_num_pos += 1
    if test_class[l] == 1:
        knn_num_neg += 1
    if knn_class[l] == test_class[l]:
        knn_acc += 1
    if knn_class[l] == 0 and test_class[l] == 0:
        knn_pos += 1
    if knn_class[l] == 1 and test_class[l] == 1:
        knn_neg += 1        

In [160]:
data = "orig"
data1 = "itg"
data2 = "resplit"
knn_accuracy = knn_acc/len(test)
knn_t_pos = knn_pos/knn_num_pos
knn_t_neg = knn_neg/knn_num_neg
print("For K-Nearest Neighbor:")
print(data2, knn_acc,"/",len(test), "(",knn_accuracy,")", knn_pos,"/",knn_num_pos, "(",knn_t_pos,")", knn_neg,"/",knn_num_neg, "(",knn_t_neg,")")

For K-Nearest Neighbor:
resplit 25 / 90 ( 0.2777777777777778 ) 19 / 19 ( 1.0 ) 6 / 71 ( 0.08450704225352113 )
